In [ ]:
import pickle
import spacy
import os
import re, string, unicodedata
import nltk
!pip install contractions
!pip install inflect
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

     |████████████████████████████████| 327kB 35.4MB/s 
     |████████████████████████████████| 266kB 51.5MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85399 sha256=2003826923a73597b34766f3fdc986d02e4b0ceff9c149e2b945236d631d5814
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def DoctoDict():
  file_Names = os.listdir("/content/drive/MyDrive/IR_ASSIGNMENT_1/stories1")
  file_Paths = []
  for i in range(len(file_Names)):
    file_Paths.append("/content/drive/MyDrive/IR_ASSIGNMENT_1/stories1/"+file_Names[i])
  corpus = {}
  for i in range(len(file_Paths)):
    with open(file_Paths[i],encoding = "latin-1") as f_input:
      corpus[file_Names[i]] = [f_input.read()]
  return corpus

In [ ]:
CORPUS = DoctoDict()
with open('CORPUS.pickle', 'wb') as handle:
  pickle.dump(CORPUS, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/content/CORPUS.pickle', 'rb') as handle:
  corpus = pickle.load(handle)

len(corpus.keys())

467

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    text=re.sub('\n',' ',text)
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

def replace_contractions(text):
    return contractions.fix(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word[0]).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append((new_word,word[1]))
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word[0].lower()
        new_words.append((new_word,word[1]))
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word[0])
        if new_word != '':
            new_words.append((new_word,word[1]))
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word[0] not in stopwords.words('english'):
            new_words.append((word[0],word[1]))
    return new_words

def get_wordnet_pos(tag):
    tag_dict = {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
    return tag_dict.get(tag,wordnet.NOUN)

def lemmatize(words):
    """Lemmatize words in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    posTagged = nltk.pos_tag(words)
    wordnetTagged = list(map(lambda x: (x[0], get_wordnet_pos(x[1][0])), posTagged))
    for word,tag in wordnetTagged:
      lemma = lemmatizer.lemmatize(word,tag)
      lemmas.append(lemma)
    return lemmas

def lemmatizeSpacy(words):
  sent = ""
  lwords = []
  for word in words:
    sent += word + " " 
  model = spacy.load("en_core_web_sm")
  tokens = model(sent)
  for i in range(len(tokens)):
    lwords.append((tokens[i].lemma_,i+1))
  # for token in tokens:
  #   lwords.append(token.lemma_)
  return lwords

def preProcess_html(fileName):
    sample = denoise_text(fileName)
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    # words = lemmatize(words)
    # words = lemmatizeSpacy(words)
    return words  

def clean_text(text):
    # text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z0-9A-Z]',' ',text)
    text=re.sub(' +',' ',text)
    return text

def preProcessotherfiles(fileName):
    sample = clean_text(fileName)
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = lemmatizeSpacy(words)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    # words = lemmatize(words)
    return words 


In [ ]:
for i in corpus.keys():
  if i.endswith(".html") or i.endswith(".header"):
    corpus[i][0] = preProcess_html(corpus[i][0])
  else:
    corpus[i][0] = preProcessotherfiles(corpus[i][0])

In [ ]:
len(corpus.keys())

467

In [ ]:
print(corpus['life.txt'])

[[('pron', 1), ('may', 2), ('well', 4), ('start', 5), ('pron', 9), ('write', 11), ('first', 13), ('pron', 16), ('writing', 20), ('sort', 21), ('mood', 23), ('secondly', 24), ('topic', 26), ('bother', 29), ('pron', 30), ('time', 33), ('pron', 35), ('long', 37), ('since', 38), ('put', 39), ('write', 41), ('sort', 43), ('text', 45), ('file', 46), ('essay', 48), ('subject', 51), ('tonight', 53), ('craving', 56), ('write', 58), ('pron', 59), ('gather', 60), ('idea', 64), ('settle', 67), ('pron', 69), ('brain', 70), ('proceed', 72), ('write', 74), ('pron', 76), ('really', 77), ('exist', 78), ('hell', 83), ('pron', 85), ('thought', 87), ('run', 88), ('dave', 90), ('mind', 92), ('pron', 94), ('sit', 95), ('wooden', 98), ('floor', 99), ('pron', 101), ('room', 102), ('pron', 103), ('late', 105), ('night', 107), ('exact', 109), ('time', 110), ('pron', 111), ('sure', 114), ('pron', 116), ('computer', 117), ('autodiale', 120), ('hack', 121), ('code', 122), ('end', 126), ('pron', 128), ('room', 129)

In [ ]:
with open('DocTerms_SpacyLemm.pickle', 'wb') as handle:
  pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/DocTerms_SpacyLemm.pickle', 'rb') as handle:
   docT = pickle.load(handle)

In [ ]:
docT.keys()

dict_keys(['tree.txt', 'consumdr.hum', 'snow.txt', 'contrad1.hum', 'candle.hum', 'spiders.txt', 'timem.hac', 'aluminum.hum', 'life.txt', 'sight.txt', 'cameloto.hum', 'beyond.hum', 'partya.txt', 'nitepeek.sto', 'blind.txt', 'altside.hum', 'abyss.txt', 't_zone.jok', 'fantas.hum', 'advsayed.txt', 'eyeargon.hum', 'elite.app', 'rocket.sf', 'corcor.hum', 'elveshoe.txt', 'ab40thv.txt', 'gold3ber.txt', 'game.txt', 'excerpt.txt', 'knuckle.txt', 'emperor3.txt', 'empnclot.txt', 'abbey.txt', 'advtthum.txt', 'archive', 'wolfcran.txt', 'immorti.hum', 'wlgirl.txt', 'cooldark.sto', 'wolf7kid.txt', 'vday.hum', 'imagin.hum', 'adv_alad.txt', 'wombat.und', 'gemdra.txt', 'aircon.txt', 'wolflamb.txt', 'testpilo.hum', 'valen', 'confilct.fun', 'narciss.txt', 'enchdup.hum', 'ladylust.hum', 'tcoa.txt', 'enginer.txt', 'encamp01.txt', 'korea.s', 'taxnovel.txt', 'greedog.txt', 'keepmodu.txt', 'vainsong.txt', 'goldgoos.txt', 'quickfix', 'goldbug.poe', 'omarsheh.txt', 'grav', 'empty.txt', 'oxfrog.txt', 'gulliver.txt